In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import time
load_dotenv(override=True)
engine = create_engine(os.environ.get("ALCHEMY_DATABASE_URL"), connect_args={'connect_timeout': 600})

## Query Function

In [2]:
#function definition for running queries
def run_query(query):
    with engine.connect() as connection:
        start_time = time.time()  
        result = pd.read_sql(query, connection)
        end_time = time.time()  
        execution_time = end_time - start_time
        return result, execution_time

## Roll Up

In [3]:
expected_roll_up_df = pd.DataFrame({
    'year': [2023, 2024, 2025],
    'total_games': [15542, 12580, 2]
})

In [4]:
# Roll Up
roll_up_query = """
SELECT D.year, COUNT(G.id) AS total_games 
FROM game_fact_table AS G 
JOIN dimDate D ON G.releaseDate = D.date 
WHERE D.year IN (2023, 2024, 2025)
GROUP BY D.year;
"""

actual_roll_up_df, roll_up_time = run_query(roll_up_query)
print("Expected Roll Up Query Result:")
print(expected_roll_up_df)
print(f"Execution Time: {roll_up_time:.4f} seconds")
print("Roll Up Query Result:")
print(actual_roll_up_df)

if actual_roll_up_df.equals(expected_roll_up_df):
    print("Test Passed: Roll Up query matches the expected result.")
else:
    print("Test Failed: Roll Up query does not match the expected result.")

print(f"Execution Time: {roll_up_time:.4f} seconds")

Expected Roll Up Query Result:
   year  total_games
0  2023        15542
1  2024        12580
2  2025            2
Execution Time: 0.0260 seconds
Roll Up Query Result:
   year  total_games
0  2023        15542
1  2024        12580
2  2025            2
Test Passed: Roll Up query matches the expected result.
Execution Time: 0.0260 seconds


## Dice

In [5]:
expected_dice_df = pd.DataFrame({
    'name': ['Multiplayer', 'PvP', 'Rugby'],  
    'tag count': [3825, 2434, 1]  
})

In [6]:
# Dice
dice_query = """
SELECT t.name, COUNT(t.name) AS 'tag count'
FROM game_fact_table g
LEFT JOIN dimcategory dc ON g.dimCategoryId = dc.dimCategoryId
LEFT JOIN category c on dc.dimCategoryId = c.groupId
LEFT JOIN dimtag dt ON dt.dimTagId = g.dimTagId
LEFT JOIN tag t ON dt.dimTagId = t.groupId
WHERE c.name = 'Online PVP' 
AND t.name IN ('Multiplayer', 'PvP', 'Rugby')
GROUP BY t.name
ORDER BY COUNT(t.name) DESC;
"""

actual_dice_df, dice_time = run_query(dice_query)
print("Expected Dice Query Result:")
print(expected_dice_df)
print(f"Execution Time: {dice_time:.4f} seconds")
print("Dice Query Result:")
print(actual_dice_df)

if actual_dice_df.equals(expected_dice_df):
    print("Test Passed: Dice query matches the expected result.")
else:
    print("Test Failed: Dice query does not match the expected result.")

print(f"Execution Time: {dice_time:.4f} seconds")

Expected Dice Query Result:
          name  tag count
0  Multiplayer       3825
1          PvP       2434
2        Rugby          1
Execution Time: 1.4350 seconds
Dice Query Result:
          name  tag count
0  Multiplayer       3825
1          PvP       2434
2        Rugby          1
Test Passed: Dice query matches the expected result.
Execution Time: 1.4350 seconds


## Drill Down

In [7]:
expected_drill_down_df = pd.DataFrame({
    'game': ['Hogwarts Legacy', 'Sons Of The Forest', 'Resident Evil 4', 'Wo Long: Fallen Dynasty', "Hero's Adventure"], 
    'releaseDate': ['2023-02-10', '2023-02-23', '2023-03-23', '2023-03-03', '2023-03-07'],
    'price': [59.99, 29.99, 59.99, 59.99, 15.83],
    'quarter': [1, 1, 1, 1, 1],
    'estimatedOwners': ['5000000 - 10000000', '2000000 - 5000000', '0 - 20000', '500000 - 1000000', '100000 - 200000'],
    'peakCCU': [872138, 405191, 153726, 59368, 28699]
})

In [8]:
# Drill Down
drill_down_query = """
SELECT name AS game, releaseDate, price, quarter, estimatedOwners, peakCCU 
FROM game_fact_table g
LEFT JOIN dimDate ddt ON ddt.date = g.releaseDate
WHERE ddt.quarter = 1 AND ddt.year = 2023 AND price > 0
AND name IN ('Hogwarts Legacy', 'Sons Of The Forest', 'Resident Evil 4', 'Wo Long: Fallen Dynasty', "Hero's Adventure")
ORDER BY peakCCU DESC;
"""

actual_drill_down_df, drilldown_time = run_query(drill_down_query)
print("Expected Drill Down Query Result:")
print(expected_drill_down_df)
print("Drill Down Query Result:")
print(actual_drill_down_df)
print(f"Execution Time: {drilldown_time:.4f} seconds")

if actual_drill_down_df.head(5).reset_index(drop=True).astype(str).equals(expected_drill_down_df.astype(str)): #used alternate comparison method as some characters were causing a 'test failed' output
    print("Test Passed: Drill Down query matches the expected result.")
else:
    print("Test Failed: Drill Down query does not match the expected result.")

Expected Drill Down Query Result:
                      game releaseDate  price  quarter     estimatedOwners  \
0          Hogwarts Legacy  2023-02-10  59.99        1  5000000 - 10000000   
1       Sons Of The Forest  2023-02-23  29.99        1   2000000 - 5000000   
2          Resident Evil 4  2023-03-23  59.99        1           0 - 20000   
3  Wo Long: Fallen Dynasty  2023-03-03  59.99        1    500000 - 1000000   
4         Hero's Adventure  2023-03-07  15.83        1     100000 - 200000   

   peakCCU  
0   872138  
1   405191  
2   153726  
3    59368  
4    28699  
Drill Down Query Result:
                      game releaseDate  price  quarter     estimatedOwners  \
0          Hogwarts Legacy  2023-02-10  59.99        1  5000000 - 10000000   
1       Sons Of The Forest  2023-02-23  29.99        1   2000000 - 5000000   
2          Resident Evil 4  2023-03-23  59.99        1           0 - 20000   
3  Wo Long: Fallen Dynasty  2023-03-03  59.99        1    500000 - 1000000   
4   

## Slice

In [9]:
expected_slice_df = pd.DataFrame({
    'game': ['Dota 2'],  
    'price': [0.0],
    'developer': ['Valve']

})

In [10]:
# Slice
slice_query = """
SELECT g.name AS game, g.price, de.name AS developer
FROM Game_Fact_Table AS g
LEFT JOIN dimDeveloper dd ON dd.dimDeveloperId = g.dimDeveloperId
LEFT JOIN developer de ON de.groupId = dd.dimDeveloperId
WHERE de.name = 'Valve' AND G.reviews !=''
ORDER BY game;
"""
actual_slice_df, slice_time = run_query(slice_query)
print("Expected Slice Query Result:")
print(expected_slice_df)
print("Drill Down Query Result:")
print(actual_slice_df)
print(f"Execution Time: {slice_time:.4f} seconds")

if actual_slice_df.equals(expected_slice_df):
    print("Test Passed: Slice query matches the expected result.")
else:
    print("Test Failed: Slice query does not match the expected result.")

Expected Slice Query Result:
     game  price developer
0  Dota 2    0.0     Valve
Drill Down Query Result:
     game  price developer
0  Dota 2    0.0     Valve
Execution Time: 0.0020 seconds
Test Passed: Slice query matches the expected result.
